In [ ]:
import requests
from bs4 import BeautifulSoup
import pandas as pd

csv_path = "/Users/suphakorninsee/Desktop/CEDT/Second_year/DataSci/DSDE-PROJ/DATA/funding.csv"
df = pd.read_csv(csv_path)

def get_country_name(geonameid):
    if pd.isna(geonameid):
        print("Skipping invalid geonameid: nan")
        return None

    geonameid = int(geonameid)
    url = f"https://www.geonames.org/{geonameid}/"
    print(f"Fetching URL: {url}")
    response = requests.get(url)
    if response.status_code == 200:
        soup = BeautifulSoup(response.content, 'html.parser')

        country_name_tag = soup.find("div", {"data-geonameid": str(geonameid), "id": "infowin"})
        if country_name_tag:
            country_name = country_name_tag.find("b").text.strip()
            print(f"Found country name: {country_name} for geonameid: {geonameid}")
            return country_name
        else:
            print(f"No country name found for geonameid: {geonameid}")
    else:
        print(f"Failed to fetch URL: {url}, Status code: {response.status_code}")
    return None

df['geonameid'] = df['agency-country'].str.extract(r'(\d+)')

print("Validating geonameid...")
df['geonameid'] = pd.to_numeric(df['geonameid'], errors='coerce')

df['country_name'] = df['geonameid'].apply(get_country_name)

output_path = "/Users/suphakorninsee/Desktop/CEDT/Second_year/DataSci/DSDE-PROJ/DATA/funding_with_countries.csv"
df.to_csv(output_path, index=False)

print(f"ผลลัพธ์ถูกบันทึกที่: {output_path}")


In [ ]:
import re
import requests
import pandas as pd

# Load the CSV file
csv_path = "/Users/suphakorninsee/Desktop/CEDT/Second_year/DataSci/DSDE-PROJ/DATA/funding.csv"
funding_data = pd.read_csv(csv_path)

# Extract the GeoName ID from the 'agency-country' column
funding_data['GeoNameID'] = funding_data['agency-country'].apply(
    lambda x: re.search(r'\d+', x).group() if isinstance(x, str) else None
)

# Function to fetch the country name from the GeoNames API
def fetch_country_name(geoname_id):
    if geoname_id:
        api_url = f"http://api.geonames.org/getJSON?geonameId={geoname_id}&username=y3nyen"
        try:
            response = requests.get(api_url)
            if response.status_code == 200:
                data = response.json()
                return data.get("name", None)
        except Exception as e:
            print(f"Error fetching data for GeoNameID {geoname_id}: {e}")
    return None

# Create a new column for country names
funding_data['country_name'] = funding_data['GeoNameID'].apply(fetch_country_name)

# Display the updated DataFrame
print(funding_data[['agency-country', 'GeoNameID', 'country_name']].head())

# Save the updated DataFrame to a new CSV if needed
output_path = "/Users/suphakorninsee/Desktop/CEDT/Second_year/DataSci/DSDE-PROJ/DATA/funding_with_countries.csv"
funding_data.to_csv(output_path, index=False)


In [21]:
import re
import requests
import pandas as pd

# Load the CSV file
csv_path = "/Users/suphakorninsee/Desktop/CEDT/Second_year/DataSci/DSDE-PROJ/DATA/funding.csv"
funding_data = pd.read_csv(csv_path)

# Extract the GeoName ID from the 'agency-country' column
funding_data['GeoNameID'] = funding_data['agency-country'].apply(
    lambda x: re.search(r'\d+', x).group() if isinstance(x, str) else None
)

# Dictionary to cache fetched country names
geoname_cache = {}

# Function to fetch the country name from the GeoNames API
def fetch_country_name(geoname_id):
    if not geoname_id:
        print(f"GeoName ID missing for entry: {geoname_id}")
        return None
    if geoname_id in geoname_cache:
        # Return cached result if available
        return geoname_cache[geoname_id]
    # Make API request if not cached
    api_url = f"http://api.geonames.org/getJSON?geonameId={geoname_id}&username=y3nyen"
    try:
        response = requests.get(api_url)
        if response.status_code == 200:
            data = response.json()
            country_name = data.get("name", None)
            if country_name:
                print(f"New country detected: {country_name}")
                geoname_cache[geoname_id] = country_name  # Cache the result
                return country_name
            else:
                print(f"No country found for GeoName ID {geoname_id}")
        else:
            print(f"API request failed for GeoName ID {geoname_id} with status code {response.status_code}")
    except Exception as e:
        print(f"Error fetching data for GeoName ID {geoname_id}: {e}")
    return None

# Map GeoName IDs to country names
funding_data['country_name'] = funding_data['GeoNameID'].map(fetch_country_name)

# Display the updated DataFrame
print(funding_data[['agency-country', 'GeoNameID', 'country_name']].head())

# Save the updated DataFrame to a new CSV if needed
output_path = "/Users/suphakorninsee/Desktop/CEDT/Second_year/DataSci/DSDE-PROJ/DATA/funding_with_countries.csv"
funding_data.to_csv(output_path, index=False)

# Optionally, save the cache for reuse
cache_path = "/Users/suphakorninsee/Desktop/CEDT/Second_year/DataSci/DSDE-PROJ/DATA/geoname_cache.json"
with open(cache_path, 'w') as cache_file:
    import json
    json.dump(geoname_cache, cache_file)


GeoName ID missing for entry: None
GeoName ID missing for entry: None
New country detected: China
New country detected: Thailand
GeoName ID missing for entry: None
GeoName ID missing for entry: None
GeoName ID missing for entry: None
GeoName ID missing for entry: None
GeoName ID missing for entry: None
New country detected: Brazil
GeoName ID missing for entry: None
GeoName ID missing for entry: None
GeoName ID missing for entry: None
GeoName ID missing for entry: None
GeoName ID missing for entry: None
GeoName ID missing for entry: None
GeoName ID missing for entry: None
GeoName ID missing for entry: None
GeoName ID missing for entry: None
GeoName ID missing for entry: None
GeoName ID missing for entry: None
GeoName ID missing for entry: None
New country detected: United States
New country detected: Germany
New country detected: Colombia
New country detected: Qatar
New country detected: Armenia
New country detected: Switzerland
New country detected: Mexico
New country detected: Türkiye

ฉันอยากทำ web srape โดยในตารางนี้คุณจะเห็น column ที่ชื่อว่า agency-country ซึ่งจะได้ลิงค์เป็น https://www.geonames.org/1814991/ คุณจะเห็นเลข 1814991 ซึ่งบ่งบอกถึง data-geonameid ของประเทศ แต่ที่ฉันอยากให้ scrape ออกมาคือชื่อของประเทศนั้น ๆ โดยในเว็ยจะมี html ตามไฟล์ message.txt คุณช่วยเขียนโค้ดเพื่อดึงชื่อมาจากลิงค์ให้หน่อย ซึ่งชื่อจะอยู่ใน data-geonameid ในส่วนของ name